In [24]:
import os
import re
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)

In [ ]:
def calculate_ALT_AD(row):
    split_AD = row.AD.split(",")[1:]
    split_AD = [int(x) for x in split_AD]
    if row.len_AD > 2:
        max_AD = max(split_AD)
        return max_AD
    else:
        #ALT_AD = row.AD.split(",")[1]
        return split_AD[0]

In [ ]:
def calculate_true_ALT(row):
    split_AD = row.AD.split(",")[1:]
    split_AD = [int(x) for x in split_AD]
    split_ALT = row.ALT.split(",")
    if row.len_AD > 2:
        max_AD = max(split_AD)
        max_index = split_AD.index(max(split_AD))
        true_ALT = split_ALT[max_index]
        return true_ALT
    else:
        #ALT_AD = row.AD.split(",")[1]
        return split_ALT[0]

In [30]:
alt_AD_ = [3, 24]

In [36]:
alt_AD_.index(24)

1

In [37]:
alt_AD_[1]

24

In [54]:
def import_annot_to_pandas(vcf_file, sep='\t'):
    """
    Parse vcf outputted by snpEFF which adds the ANN field
    Dependences: calculate_ALT_AD
                calculate_true_ALT
    """
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
    
    if first_line.endswith('VCFv4.2'):
        
        #Use first line as header
        dataframe = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
        sample = dataframe.columns[-1]
        dataframe.rename(columns={sample:'sample'}, inplace=True)
        
        ann_head = ["Allele","Annotation","Annotation_Impact","Gene_Name",
                    "Gene_ID","Feature_Type","Feature_ID","Transcript_BioType",
                    "Rank","HGVS.c","HGVS.p","cDNA.pos / cDNA.length",
                    "CDS.pos / CDS.length","AA.pos / AA.length","Distance",
                    "ERRORS / WARNINGS / INFO"]
        
        for index, data_row in dataframe.iterrows():
            info_fields = re.findall(r';*([a-zA-Z]{1,20})=', data_row.INFO)
            info_values = re.findall(r'-?\d+\.?\d*e?[+-]?\d{0,2}', data_row.INFO)
            ann_values_re = re.search(r'ANN=(.*)\|(.*)', data_row.INFO)
            all_ann_values = ann_values_re.group(1)
            ann_values = all_ann_values.split(",")[0].split("|")[:16]
            
            format_fields = data_row['FORMAT'].split(":")
            format_values = data_row['sample'].split(":")
                                    
            for ifield, ivalue in zip(info_fields,info_values):
                dataframe.loc[index,ifield] = ivalue
                
            for ffield, fvalue in zip(format_fields,format_values):
                dataframe.loc[index,ffield] = fvalue
            
            dataframe.loc[index,'ANN'] = all_ann_values
            
            for ann_field,ann_value in zip(ann_head, ann_values):
                dataframe.loc[index,ann_field] = ann_value
            
        dataframe.rename(columns={'AF':'af'}, inplace=True)
        
        dataframe['len_AD'] = dataframe['AD'].str.split(",").str.len()
        dataframe['REF_AD'] = dataframe['AD'].str.split(",").str[0]

        dataframe['ALT_AD'] = dataframe.apply(calculate_ALT_AD, axis=1)
        dataframe['ALT'] = dataframe.apply(calculate_true_ALT, axis=1)

        dataframe[['gt0','gt1']] = dataframe['GT'].str.split(r'[/|\|]', expand=True)

        dataframe['HGVS.c'] = dataframe['HGVS.c'].str.split(".").str[-1]
        dataframe['HGVS.p'] = dataframe['HGVS.p'].str.split(".").str[-1]
        dataframe['Gene length'] = dataframe['CDS.pos / CDS.length'].str.split("/").str[-1]
        dataframe['AA length'] = dataframe['AA.pos / AA.length'].str.split("/").str[-1]
        dataframe['AA length'] = dataframe['AA length'].replace('', 0)
        dataframe['HGVS.p'] = dataframe['HGVS.p'].replace('', 'None')
                
        to_float = ['QUAL', 'AC', 'af', 'AN', 'BaseQRankSum', 'DP', 'ExcessHet', 'FS',
       'MLEAC', 'MLEAF', 'MQ', 'MQRankSum', 'QD', 'ReadPosRankSum', 'SOR','GQ','ALT_AD', 'REF_AD']
        
        to_int = ['POS', 'len_AD', 'gt0', 'gt1']
        
        to_str = ['#CHROM','REF','ALT', 'FILTER']
        
        for column in dataframe.columns:
            if column in to_float:
                dataframe[column] = dataframe[column].astype(float)
                
        for column in dataframe.columns:
            if column in to_int:
                dataframe[column] = dataframe[column].astype(int)
                
        for column in dataframe.columns:
            if column in to_str:
                dataframe[column] = dataframe[column].astype(str)
                
        dataframe['dp'] = (dataframe['REF_AD'] + dataframe['ALT_AD'])
        dataframe['aF'] = dataframe['REF_AD']/dataframe['dp']
        dataframe['AF'] = dataframe['ALT_AD']/dataframe['dp']
        

    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return dataframe

In [55]:
annot_vcf = "/home/laura/ANALYSIS/VARIANT_CALLING/MTB_ANC/Annotation/Gui-16.combined.hf.annot"

high_confidence_file = "/home/laura/DEVELOP/SNPTB/annotation/resistance/dict_position_resistance_high_conf.txt"
resistance_file_V1 = "/home/laura/DEVELOP/SNPTB/annotation/resistance/dict_position_resistance_v1.txt"
resistance_file_v2 = "/home/laura/DEVELOP/SNPTB/annotation/resistance/dict_position_resistance_v2_inf.txt"
ecoli_annot_file = "/home/laura/DEVELOP/SNPTB/annotation/resistance/EColi.txt"

In [70]:
vcf_df = import_annot_to_pandas(annot_vcf)

In [74]:
vcf_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,BaseQRankSum,DP,ExcessHet,FS,InbreedingCoeff,MQ,MQRankSum,QD,ReadPosRankSum,SOR,ANN,GT,AD,GQ,PL,Allele,Annotation,Annotation_Impact,Gene_Name,Gene_ID,Feature_Type,Feature_ID,Transcript_BioType,Rank,HGVS.c,HGVS.p,cDNA.pos / cDNA.length,CDS.pos / CDS.length,AA.pos / AA.length,Distance,PGT,PID,PS,LOF,ERRORS / WARNINGS / INFO,len_AD,REF_AD,ALT_AD,gt0,gt1,Gene length,AA length,dp,aF,AF
0,Chromosome,2532,.,C,T,432235.58,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.77;DP=11;Exce...,GT:AD:DP:GQ:PL,"1/1:0,11:11:33:340,33,0",2.0,1.0,2.0,1.770,11.0,0.0,0.0,0.9914,60.0,0.0,34.08,0.274,0.013,T|synonymous_variant|LOW|dnaN|Rv0002|transcrip...,1/1,"0,11",33.0,"340,33,0",T,synonymous_variant,LOW,dnaN,Rv0002,transcript,CCP42724.1,protein_coding,1/1,481C>T,Leu161Leu,481/1209,481/1209,161/402,,NaN,NaN,NaN,NaN,NaN,2,0.0,11.0,1,1,1209,402,11.0,0.0,1.0
1,Chromosome,9143,.,C,T,891226.50,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.67;DP=22;Exce...,GT:AD:DP:GQ:PL,"1/1:0,22:22:66:669,66,0",2.0,1.0,2.0,1.670,22.0,0.0,0.0,0.9672,60.0,0.0,34.29,0.000,0.794,T|synonymous_variant|LOW|gyrA|Rv0006|transcrip...,1/1,"0,22",66.0,"669,66,0",T,synonymous_variant,LOW,gyrA,Rv0006,transcript,CCP42728.1,protein_coding,1/1,1842C>T,Ile614Ile,1842/2517,1842/2517,614/838,,NaN,NaN,NaN,NaN,NaN,2,0.0,22.0,1,1,2517,838,22.0,0.0,1.0
2,Chromosome,13460,.,G,A,883588.72,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.16;DP=21;Exce...,GT:AD:DP:GQ:PL,"1/1:0,21:21:62:618,62,0",2.0,1.0,2.0,1.160,21.0,0.0,0.0,1.0000,60.0,0.0,33.59,0.813,0.011,A|synonymous_variant|LOW|Rv0010c|Rv0010c|trans...,1/1,"0,21",62.0,"618,62,0",A,synonymous_variant,LOW,Rv0010c,Rv0010c,transcript,CCP42732.1,protein_coding,1/1,99C>T,Asp33Asp,99/426,99/426,33/141,,NaN,NaN,NaN,NaN,NaN,2,0.0,21.0,1,1,426,141,21.0,0.0,1.0
3,Chromosome,14251,.,G,A,276717.42,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.716e+00;DP=2...,GT:AD:DP:GQ:PL,"1/1:0,23:23:69:662,69,0",2.0,1.0,2.0,-1.716,23.0,-0.0,0.0,0.9967,60.0,0.0,32.00,-0.617,1.043,A|missense_variant|MODERATE|Rv0012|Rv0012|tran...,1/1,"0,23",69.0,"662,69,0",A,missense_variant,MODERATE,Rv0012,Rv0012,transcript,CCP42734.1,protein_coding,1/1,163G>A,Asp55Asn,163/789,163/789,55/262,,NaN,NaN,NaN,NaN,NaN,2,0.0,23.0,1,1,789,262,23.0,0.0,1.0
4,Chromosome,14401,.,G,A,744278.57,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.400e-02;DP=2...,GT:AD:DP:GQ:PL,"1/1:0,27:27:81:809,81,0",2.0,1.0,2.0,-0.014,27.0,0.0,0.0,0.9075,60.0,0.0,33.12,0.043,0.516,A|synonymous_variant|LOW|Rv0012|Rv0012|transcr...,1/1,"0,27",81.0,"809,81,0",A,synonymous_variant,LOW,Rv0012,Rv0012,transcript,CCP42734.1,protein_coding,1/1,313G>A,Lys105Lys,313/789,313/789,105/262,,NaN,NaN,NaN,NaN,NaN,2,0.0,27.0,1,1,789,262,27.0,0.0,1.0


In [75]:
def file_to_dict(file_format):
    formatted_dict = {}
    with open (file_format, 'r') as f:
        for line in f:
            formatted_dict[int(line.split(":")[0])] = line.split(":")[1].strip().split(",")
    return formatted_dict

In [63]:
dict_res_v1 = file_to_dict(resistance_file_V1)
dict_res_v2 = file_to_dict(resistance_file_v2)
dict_high_conf = file_to_dict(high_confidence_file)
dict_ecoli_annot = file_to_dict(ecoli_annot_file)

In [76]:
dict_res_v1

{6575: ['fluoroquinolones (FQ)', 'T'],
 6620: ['fluoroquinolones (FQ)', 'C', 'A'],
 6621: ['fluoroquinolones (FQ)', 'C'],
 6734: ['fluoroquinolones (FQ)', 'G', 'A'],
 6735: ['fluoroquinolones (FQ)', 'C'],
 6736: ['fluoroquinolones (FQ)', 'G'],
 6737: ['fluoroquinolones (FQ)', 'C'],
 6738: ['fluoroquinolones (FQ)', 'A', 'T'],
 6741: ['fluoroquinolones (FQ)', 'T'],
 6742: ['fluoroquinolones (FQ)', 'T'],
 6749: ['fluoroquinolones (FQ)', 'A'],
 6750: ['fluoroquinolones (FQ)', 'T'],
 7563: ['fluoroquinolones (FQ)', 'T'],
 7564: ['fluoroquinolones (FQ)', 'C'],
 7566: ['fluoroquinolones (FQ)', 'A'],
 7570: ['fluoroquinolones (FQ)', 'T'],
 7572: ['fluoroquinolones (FQ)', 'C'],
 7581: ['fluoroquinolones (FQ)', 'C', 'A', 'T'],
 7582: ['fluoroquinolones (FQ)', 'G', 'C', 'T'],
 575729: ['ethionamide (ETH)', 'T'],
 576164: ['ethionamide (ETH)', 'T'],
 576242: ['ethionamide (ETH)', 'T'],
 576338: ['ethionamide (ETH)', 'T'],
 576414: ['ethionamide (ETH)', 'A'],
 576429: ['ethionamide (ETH)', 'C'],
 7

In [84]:
def add_resistance_snp(vcf_df, dict_high_confidence=dict_high_conf, dict_resistance_position=dict_res_v1,
                       dict_resistance_v2=dict_res_v2, dict_ecoli_annot=dict_ecoli_annot):
    
    """
    TODO:Function for REs_v1, Res_V2 and ecoli
    """
    
    vcf_df['Resistance'] = np.nan
    vcf_df['Resistance_inf'] = np.nan
    vcf_df['ecoli_annot'] = np.nan

    for index, _ in vcf_df.iterrows():
        position = int(vcf_df.loc[index,'POS'])
        alt_nucleotide = str(vcf_df.loc[index,'ALT'])
        
        if position in dict_resistance_position.keys():
            #Check position in resistance dict
            #Create a list with all possible nucleotydes in each position
            nucleotides = dict_resistance_position[position][1:]
            if alt_nucleotide in nucleotides:
                resistance = dict_resistance_position[int(position)][0] #Resist name
                vcf_df.loc[index, 'Resistance'] = resistance
                #Evaluate High confidence (1.Position; 2. Nucleotide; 3. yes value)
                if position in dict_high_confidence.keys() and \
                (alt_nucleotide in dict_high_confidence[int(position)][1:] and \
                dict_high_confidence[int(position)][0] == 'no'):
                    vcf_df.loc[index,'Resistance'] = resistance + "*"
                    
        if position in dict_resistance_v2.keys():
            #Check position in resistance dict
            #Create a list with all possible nucleotydes in each position
            nucleotides = dict_resistance_v2[position][1:]
            if alt_nucleotide in nucleotides:
                resistance_2 = dict_resistance_v2[int(position)][0] #Resist name
                vcf_df.loc[index, 'Resistance_inf'] = resistance_2
                
        if position in dict_ecoli_annot.keys():
            #Check position in resistance dict
            #Create a list with all possible nucleotydes in each position
            nucleotides = dict_ecoli_annot[position][1:]
            if alt_nucleotide in nucleotides:
                resistance_eco = dict_ecoli_annot[int(position)][0] #Resist name
                vcf_df.loc[index, 'ecoli_annot'] = resistance_eco
                

In [85]:
add_resistance_snp(vcf_df)

In [86]:
vcf_df[vcf_df.Resistance_inf.notnull()]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,BaseQRankSum,DP,ExcessHet,FS,InbreedingCoeff,MQ,MQRankSum,QD,ReadPosRankSum,SOR,ANN,GT,AD,GQ,PL,Allele,Annotation,Annotation_Impact,Gene_Name,Gene_ID,Feature_Type,Feature_ID,Transcript_BioType,Rank,HGVS.c,HGVS.p,cDNA.pos / cDNA.length,CDS.pos / CDS.length,AA.pos / AA.length,Distance,PGT,PID,PS,LOF,ERRORS / WARNINGS / INFO,len_AD,REF_AD,ALT_AD,gt0,gt1,Gene length,AA length,dp,aF,AF,Resistance,Resistance_inf,ecoli_annot
301,Chromosome,761155,.,C,T,50054.58,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.252e+00;DP=3...,GT:AD:DP:GQ:PL,"1/1:0,38:38:99:1049,112,0",2.0,1.0,2.0,-1.252,38.0,0.0000,0.000,0.9595,60.00,0.00,32.21,-0.836,0.548,T|missense_variant|MODERATE|rpoB|Rv0667|transc...,1/1,"0,38",99.0,"1049,112,0",T,missense_variant,MODERATE,rpoB,Rv0667,transcript,CCP43410.1,protein_coding,1/1,1349C>T,Ser450Leu,1349/3519,1349/3519,450/1172,,NaN,NaN,NaN,NaN,NaN,2,0.0,38.0,1,1,3519,1172,38.0,0.000000,1.000000,rifampicin (RMP),RIFABUTIN,Ser531Leu
302,Chromosome,761244,.,A,G,4466.40,PASS,AC=2;AF=1.00;AN=2;DP=32;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,32:32:95:1017,95,0",2.0,1.0,2.0,NaN,32.0,0.0000,0.000,0.9213,59.94,NaN,30.38,NaN,1.125,G|missense_variant|MODERATE|rpoB|Rv0667|transc...,1/1,"0,32",95.0,"1017,95,0",G,missense_variant,MODERATE,rpoB,Rv0667,transcript,CCP43410.1,protein_coding,1/1,1438A>G,Ile480Val,1438/3519,1438/3519,480/1172,,NaN,NaN,NaN,NaN,NaN,2,0.0,32.0,1,1,3519,1172,32.0,0.000000,1.000000,NaN,RIFAMPICIN,NaN
552,Chromosome,1472358,.,C,T,983.38,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.296e+00;DP=...,GT:AD:DP:GQ:PL,"0/1:2,20:22:11:485,0,11",1.0,0.5,2.0,-2.296,22.0,0.0342,8.015,0.5143,59.86,3.32,23.41,1.090,1.012,T|non_coding_transcript_exon_variant|MODIFIER|...,0/1,"2,20",11.0,"485,0,11",T,non_coding_transcript_exon_variant,MODIFIER,rrs,EBG00000313325,transcript,EBG00000313325-1.1,rRNA,1/1,513C>T,None,,,,,NaN,NaN,NaN,NaN,NaN,2,2.0,20.0,0,1,,0,22.0,0.090909,0.909091,streptomycin (SM)*,streptomycin (SM),NaN
642,Chromosome,1673425,.,C,T,34393.43,PASS,AC=2;AF=1.00;AN=2;DP=37;ExcessHet=-0.0000;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,37:37:99:1064,110,0",2.0,1.0,2.0,NaN,37.0,-0.0000,0.000,0.9986,60.00,NaN,31.99,NaN,0.817,T|intergenic_region|MODIFIER|Rv1482c-fabG1|Rv1...,1/1,"0,37",99.0,"1064,110,0",T,intergenic_region,MODIFIER,Rv1482c-fabG1,Rv1482c-Rv1483,intergenic_region,Rv1482c-Rv1483,,,1673425C>T,None,,,,,NaN,NaN,NaN,NaN,NaN,2,0.0,37.0,1,1,,0,37.0,0.000000,1.000000,isoniazid (INH),ETHIONAMIDE,NaN
1108,Chromosome,3260301,.,A,G,319846.88,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=0.934;DP=19;Exc...,GT:AD:DP:GQ:PL,"1/1:0,19:19:57:619,57,0",2.0,1.0,2.0,0.934,19.0,-0.0000,0.000,0.9999,60.00,0.00,30.46,0.347,0.101,G|synonymous_variant|LOW|ppsC|Rv2933|transcrip...,1/1,"0,19",57.0,"619,57,0",G,synonymous_variant,LOW,ppsC,Rv2933,transcript,CCP45736.1,protein_coding,1/1,4617A>G,Gly1539Gly,4617/6567,4617/6567,1539/2188,,NaN,NaN,NaN,NaN,NaN,2,0.0,19.0,1,1,6567,2188,19.0,0.000000,1.000000,NaN,PYRAZINAMIDE,NaN
1335,Chromosome,4038287,.,G,A,305677.86,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.76;DP=30;Exce...,GT:AD:DP:GQ:PL,"1/1:0,30:30:89:812,89,0",2.0,1.0,2.0,1.760,30.0,-0.0000,0.000,0.9974,60.00,0.00,33.00,0.464,0.223,A|synonymous_variant|LOW|clpC1|Rv3596c|transcr...,1/1,"0,30",89.0,"812,89,0",A,synonymous_variant,LOW,clpC1,Rv3596c,transcript,CCP46419.1,protein_coding,1/1,2418C>T,Asn806Asn,2418/2547,2418/2547,806/848,,NaN,NaN,NaN,NaN,NaN,2,0.0,30.0,1,1,2547,848,30.0,0.000000,1.000000,NaN,PYRAZINAMIDE,NaN
1405,Chromosome,4243190,.,G,C,3721.24,PASS,AC=2;AF=1.00;AN=2;DP=17;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,17:17:51:547,51,0",2.0,1.0,2.0,NaN,17.0,0.0000,0.000,0.9993,60.00,NaN,30.50,NaN,0.795,C|intergenic_region|MODIFIER|embC-embA|Rv3793-...,1/1,"0,17",51.0,"547,51,0",C,intergenic_region,MODIFIER,embC-embA,Rv3793-Rv3794,intergenic_region,Rv3793-Rv3794,,,4243190G>C,None,,,,,NaN,NaN,NaN,NaN,NaN,2,0.0,17.0,1,1,,0,17.0,0.000000,1.000000,NaN,ETHAMBUTOL,NaN
